# `ipython`
> Set of utility functions to be used in Jupyter and Jupyter Lab notebooks.


In [ ]:
#|default_exp ipython

In [ ]:
#| export
from __future__ import annotations
from fastcore.test import test_fail
from functools import wraps
from IPython.core.getipython import get_ipython
from IPython.display import display, Markdown, display_markdown
from pathlib import Path
from typing import Any, List, Callable, Optional
from ecutilities.core import safe_path, path_to_parent_dir, is_type, CurrentMachine

import numpy as np
import pandas as pd
import sys
import subprocess
import warnings

In [ ]:
#| hide
from nbdev import show_doc, nbdev_export

# System and CLI

In [ ]:
#| export
def run_cli(cmd:str = 'ls -l'   # command to execute in the cli
           ):
    """Runs a cli command from jupyter notebook and print the shell output message
    
    Uses subprocess.run with passed command to run the cli command"""
    p = subprocess.run(cmd, stdout=subprocess.PIPE, shell=True)
    print(str(p.stdout, 'utf-8'))

In [ ]:
run_cli('pwd')

/home/vtec/projects/ec-packages/ecutilities/nbs-dev



# Notebook setup

In [ ]:
#| export
def nb_setup(
    autoreload:bool = True,       # True to set autoreload in this notebook
    paths:List[str|Path] = None   # Paths to add to the path environment variable
    ):
    """Use in first cell of notebook to set autoreload, and add system paths
    
    Always add a path to the directoruy 'src' if `srs` directory exists at the same level as the `nbs` directory.

    When the notebook is not located in a tree including the name `nbs`, `src` directory is searched at the same level
    as the directory in which the notebook is located.
    """
    #  Handle paths
    #  Add src if it exists
    nbs_root = path_to_parent_dir('nbs')
    p2src = (nbs_root / '../src').resolve().absolute()
    if p2src.is_dir():
        p = str(p2src.absolute())
        if p not in sys.path:
            sys.path.insert(0, p)
            print(f"Added path: {p2src.absolute()}")
    # Add passed paths
    if paths:
        for p in paths:
            if isinstance(p, Path): p = str(p.resolve().absolute())
            if p not in sys.path:
                sys.path.insert(1, p)
                print(f"Added path: {p}")

#   Setup auto reload
    if autoreload:
        ipshell = get_ipython()
        ipshell.run_line_magic('load_ext',  'autoreload')
        ipshell.run_line_magic('autoreload', '2')
        print('Set autoreload mode')

In [ ]:
show_doc(nb_setup)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/ipython.py#L32){target="_blank" style="float:right; font-size:smaller"}

### nb_setup

>      nb_setup (autoreload:bool=True, paths:List[str|pathlib.Path]=None)

Use in first cell of notebook to set autoreload, and add system paths

Always add a path to the directoruy 'src' if `srs` directory exists at the same level as the `nbs` directory.

When the notebook is not located in a tree including the name `nbs`, `src` directory is searched at the same level
as the directory in which the notebook is located.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| autoreload | bool | True | True to set autoreload in this notebook |
| paths | List[str \| Path] | None | Paths to add to the path environment variable |

By default, `ipython.nb_setup()` 
- loads and set `autoreload`
- adds a path to a directory named `src` when it exists at the same level as where the notebook directory is located. It no such `src` directory exists, no path is added

`ipython.nb_setup` assumes the following file structure:

```
    project_directory
          |--- nbs
          |     | --- current_nb.ipynb
          |     | --- ...
          |
          |--- src
          |     | --- module_to_import.py
          |     | --- ...
          |
          |--- data
          |     |
          |     | ...
```

For other file structure, specify paths as a `list` of `Path`

Before running `nb_setup`, `sys.path` does not include the path to the local source directory. After running it, it will be added, unless the directory does not exist.

In [ ]:
sys.path

['/home/vtec/projects/ec-packages/ecutilities/nbs-dev',
 '/home/vtec/miniconda3/envs/ecutils/lib/python310.zip',
 '/home/vtec/miniconda3/envs/ecutils/lib/python3.10',
 '/home/vtec/miniconda3/envs/ecutils/lib/python3.10/lib-dynload',
 '',
 '/home/vtec/miniconda3/envs/ecutils/lib/python3.10/site-packages',
 '/home/vtec/miniconda3/envs/ecutils/lib/python3.10/site-packages/PyQt5_sip-12.11.0-py3.10-linux-x86_64.egg',
 '/home/vtec/projects/ec-packages/ecutilities']

In [ ]:
nb_setup(autoreload=False)

Added path: /home/vtec/projects/ec-packages/ecutilities/src


In [ ]:
sys.path

['/home/vtec/projects/ec-packages/ecutilities/src',
 '/home/vtec/projects/ec-packages/ecutilities/nbs-dev',
 '/home/vtec/miniconda3/envs/ecutils/lib/python310.zip',
 '/home/vtec/miniconda3/envs/ecutils/lib/python3.10',
 '/home/vtec/miniconda3/envs/ecutils/lib/python3.10/lib-dynload',
 '',
 '/home/vtec/miniconda3/envs/ecutils/lib/python3.10/site-packages',
 '/home/vtec/miniconda3/envs/ecutils/lib/python3.10/site-packages/PyQt5_sip-12.11.0-py3.10-linux-x86_64.egg',
 '/home/vtec/projects/ec-packages/ecutilities']

We also can add other specific paths:

In [ ]:
path_to_add = str(Path('../nbs').resolve().absolute())
nb_setup(autoreload=False, paths=[path_to_add])

Added path: /home/vtec/projects/ec-packages/ecutilities/nbs


In [ ]:
sys.path

['/home/vtec/projects/ec-packages/ecutilities/src',
 '/home/vtec/projects/ec-packages/ecutilities/nbs',
 '/home/vtec/projects/ec-packages/ecutilities/nbs-dev',
 '/home/vtec/miniconda3/envs/ecutils/lib/python310.zip',
 '/home/vtec/miniconda3/envs/ecutils/lib/python3.10',
 '/home/vtec/miniconda3/envs/ecutils/lib/python3.10/lib-dynload',
 '',
 '/home/vtec/miniconda3/envs/ecutils/lib/python3.10/site-packages',
 '/home/vtec/miniconda3/envs/ecutils/lib/python3.10/site-packages/PyQt5_sip-12.11.0-py3.10-linux-x86_64.egg',
 '/home/vtec/projects/ec-packages/ecutilities']

In [ ]:
#| export
def install_code_on_cloud(
    package_name:str, # project package name, e.g. metagentools or git+https://github.com/repo.git@main
    quiet:bool=False # install quietly with Trud
):
    """pip install the project code package, when nb is running in the cloud."""
    
    machine = CurrentMachine()

    if machine.is_colab:
        CLOUD = True
        print('The notebook is running on colab.', end=' ')
        print(f'Will install {package_name}')
    elif machine.is_kaggle:
        CLOUD = True
        print('The notebook is running on kaggle.', end=' ')
        print(f'Will install {package_name}')
    elif machine.is_local:
        CLOUD = False
        print('The notebook is running locally, will not automatically install project code')
    else:
        CLOUD = True
        print('The notebook is running on a cloud VM or the machine was not registered as local')
        print(f'Will install {package_name}')

    if CLOUD:
        print(f'Installing project code {package_name}')
        cmd = f"pip install -{'qq' if quiet else ''}U {package_name}"
        run_cli(cmd)
        print((f"{package_name} is installed."))

In [ ]:
show_doc(install_code_on_cloud)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/ipython.py#L65){target="_blank" style="float:right; font-size:smaller"}

### install_code_on_cloud

>      install_code_on_cloud (package_name:str, quiet:bool=False)

pip install the project code package, when nb is running in the cloud.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| package_name | str |  | project package name, e.g. metagentools or git+https://github.com/repo.git@main |
| quiet | bool | False | install quietly with Trud |

When using colab, kaggle or another cloud VM, specicif code must be installed every time from the Python Package Index (PyPI) or its GitHub repo.

When running locally, the project code should be pre-installed as part of the environment

In [ ]:
install_code_on_cloud(package_name='metagentools');

The notebook is running locally, will not automatically install project code


# Improve output cell formats

In [ ]:
#| export
def display_mds(
    *strings:str|tuple[str] # any number of strings with text in markdown format
):
    """Display one or several strings formatted in markdown format"""
    for string in strings:
        display_markdown(Markdown(data=string))

In [ ]:
show_doc(display_mds)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/ipython.py#L96){target="_blank" style="float:right; font-size:smaller"}

### display_mds

>      display_mds (*strings:str|tuple[str])

Display one or several strings formatted in markdown format

In [ ]:
display_mds('**bold** and _italic_')

**bold** and _italic_

In [ ]:
display_mds('**bold** and _italic_',
            '- bullet',
            '- bullet',
            '> Note: this is a note'
)

**bold** and _italic_

- bullet

- bullet

> Note: this is a note

In [ ]:
#| export
def display_dfs(*dfs:pd.DataFrame       # any number of Pandas DataFrames
               ):
    """Display one or several `pd.DataFrame` in a single cell output"""
    for df in dfs:
        display(df)

In [ ]:
show_doc(display_dfs)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/ipython.py#L104){target="_blank" style="float:right; font-size:smaller"}

### display_dfs

>      display_dfs (*dfs:pandas.core.frame.DataFrame)

Display one or several `pd.DataFrame` in a single cell output

In [ ]:
df1 = pd.DataFrame(data=np.random.normal(size=(10,5)))
df2 = pd.DataFrame(data=np.random.normal(size=(20,10)))

display_dfs(df1.head(3), df2.head(3))

,0,1,2,3,4
0,-0.404293,-1.959411,-0.136628,0.727151,-0.595115
1,2.404413,1.958637,-0.513681,-1.408594,0.950348
2,1.684596,-2.020807,0.395364,2.704780,-1.106685


,0,1,2,3,4,5,6,7,8,9
0,-2.236748,-1.984643,1.505316,-0.778855,0.461898,-1.497143,-0.533513,0.450035,0.119140,-0.567164
1,1.016743,-0.181036,-0.593569,-1.842810,1.858056,0.480069,0.266183,1.229341,0.365643,-0.231381
2,-0.567689,1.072291,0.484437,-0.224742,0.624904,-1.132879,1.338664,-0.931461,-0.035472,-0.873919


In [ ]:
#| export
class pandas_nrows_ncols:
    """Context manager that sets the max number of rows and cols to apply to any output within the context"""
    def __init__(
        self, 
        nrows:int|None=None, # max number of rows to show; show all rows if `None`
        ncols:int|None=None, # max number of columns to show; show all columns if `None`
    ):
        self.nrows = nrows
        self.ncols = ncols
    
    def __enter__(self):
        self.max_rows = pd.options.display.max_rows
        self.max_cols = pd.options.display.max_columns
        pd.options.display.max_rows = self.nrows
        pd.options.display.max_columns = self.ncols
        return self.max_rows, self.max_cols

    def __exit__(self, exc_type, exc_value, exc_tb):
        pd.options.display.max_rows = self.max_rows
        pd.options.display.max_columns = self.max_cols

In [ ]:
show_doc(pandas_nrows_ncols)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/ipython.py#L111){target="_blank" style="float:right; font-size:smaller"}

### pandas_nrows_ncols

>      pandas_nrows_ncols (nrows:int|None=None, ncols:int|None=None)

Context manager that sets the max number of rows and cols to apply to any output within the context

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| nrows | int \| None | None | max number of rows to show; show all rows if `None` |
| ncols | int \| None | None | max number of columns to show; show all columns if `None` |

With no context manager, the pandas object are displayed with a maximum of 60 rows and 20 columns.

In [ ]:
df = pd.DataFrame(np.random.randint(low=0, high=100, size=(3,50)))
display(df)

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,78,57,39,53,91,46,7,0,83,92,...,92,49,11,69,56,93,51,26,76,81
1,31,25,45,99,82,52,4,97,75,59,...,19,12,78,55,11,75,97,78,97,64
2,38,91,62,45,95,97,97,35,46,24,...,6,64,31,59,0,83,17,21,54,47


Using the context manager, all rows and columns will be displayed

In [ ]:
with pandas_nrows_ncols():
    display(df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,78,57,39,53,91,46,7,0,83,92,0,30,68,6,14,26,28,94,1,15,83,64,78,19,26,91,64,66,26,66,38,24,61,32,70,19,97,69,97,2,92,49,11,69,56,93,51,26,76,81
1,31,25,45,99,82,52,4,97,75,59,43,63,0,23,86,25,62,76,44,31,1,68,3,60,64,93,91,13,90,96,96,32,74,74,53,29,11,45,48,76,19,12,78,55,11,75,97,78,97,64
2,38,91,62,45,95,97,97,35,46,24,89,17,95,89,57,85,18,68,94,91,40,77,66,95,98,19,65,82,89,45,75,18,35,60,65,53,7,37,96,74,6,64,31,59,0,83,17,21,54,47


It is also possible to specifically define the number of rows and columns to display

In [ ]:
with pandas_nrows_ncols(nrows=2, ncols=6):
    display(df)

,0,1,2,...,47,48,49
0,78,57,39,...,26,76,81
...,...,...,...,...,...,...,...
2,38,91,62,...,21,54,47


In [ ]:
with pandas_nrows_ncols(2,6):
    print(df)

    0   1   2   ...  47  48  49
0   78  57  39  ...  26  76  81
..  ..  ..  ..  ...  ..  ..  ..
2   38  91  62  ...  21  54  47

[3 rows x 50 columns]


> ### Technical background
> 
> the context manager uses pandas's [`options API`](https://pandas.pydata.org/docs/user_guide/options.html)

In [ ]:
pd.options.display.max_rows, pd.options.display.max_columns

(60, 20)

In [ ]:
pd.get_option('display.max_rows'), pd.get_option('display.max_columns')

(60, 20)

In [ ]:
pd.describe_option('display.max_rows')

display.max_rows : int
    If max_rows is exceeded, switch to truncate view. Depending on
    `large_repr`, objects are either centrally truncated or printed as
    a summary view. 'None' value means unlimited.

    In case python/IPython is running in a terminal and `large_repr`
    equals 'truncate' this can be set to 0 and pandas will auto-detect
    the height of the terminal and print a truncated object which fits
    the screen height. The IPython notebook, IPython qtconsole, or
    IDLE do not run in a terminal and hence it is not possible to do
    correct auto-detection.
    [default: 60] [currently: 60]


In [ ]:
pd.options.display.max_rows = 10
pd.reset_option('display.max_rows')
pd.options.display.max_rows

60

In [ ]:
#| export
def display_full_df(
    df:pd.DataFrame|pd.Series,  # `DataFrame` or `Series` to display
):
    """Display a pandas `DataFrame` or `Series` showing all rows and columns"""
    if is_type(df, pd.DataFrame, raise_error=False) or is_type(df, pd.Series, raise_error=False):
        with pandas_nrows_ncols():
            display(df)
    else:
        raise TypeError(f"df must me a pandas `DataFrame` or `Series`, not a {type(df)}")

In [ ]:
show_doc(display_full_df)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/ipython.py#L133){target="_blank" style="float:right; font-size:smaller"}

### display_full_df

>      display_full_df
>                       (df:pandas.core.frame.DataFrame|pandas.core.series.Serie
>                       s)

Display a pandas `DataFrame` or `Series` showing all rows and columns

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| df | pd.DataFrame \| pd.Series | `DataFrame` or `Series` to display |

In [ ]:
df = pd.DataFrame(np.random.randint(low=0, high=100, size=(3,50)))
df

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,11,70,3,84,10,33,54,77,5,70,...,12,77,41,30,70,70,73,84,27,86
1,8,93,0,34,68,74,71,36,69,33,...,58,98,42,91,66,30,18,66,51,89
2,24,75,2,94,63,88,49,83,91,14,...,15,17,24,74,19,64,57,92,23,70


In [ ]:
display_full_df(df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,11,70,3,84,10,33,54,77,5,70,81,24,93,56,3,74,86,92,7,49,87,33,28,53,82,43,76,52,55,28,99,0,26,1,51,67,68,87,56,64,12,77,41,30,70,70,73,84,27,86
1,8,93,0,34,68,74,71,36,69,33,79,75,78,42,72,32,50,60,91,0,86,26,13,37,23,48,74,38,52,77,71,30,3,48,22,46,2,92,38,48,58,98,42,91,66,30,18,66,51,89
2,24,75,2,94,63,88,49,83,91,14,16,65,65,16,37,92,67,90,47,40,39,50,7,61,93,18,43,86,6,25,39,39,91,75,43,13,11,88,6,91,15,17,24,74,19,64,57,92,23,70


In [ ]:
#| hide
display_full_df(df.loc[0, :].T)

0     11
1     70
2      3
3     84
4     10
5     33
6     54
7     77
8      5
9     70
10    81
11    24
12    93
13    56
14     3
15    74
16    86
17    92
18     7
19    49
20    87
21    33
22    28
23    53
24    82
25    43
26    76
27    52
28    55
29    28
30    99
31     0
32    26
33     1
34    51
35    67
36    68
37    87
38    56
39    64
40    12
41    77
42    41
43    30
44    70
45    70
46    73
47    84
48    27
49    86
Name: 0, dtype: int64

In [ ]:
msg = 'should raise a TypeError'
contains = 'df must me a pandas `DataFrame` or `Series`'

test_fail(display_full_df, args=['a string'], msg=msg, contains=contains)

In [ ]:
#| hide
nbdev_export()